<div class="alert alert-info"><h1>Natural Language Processing with <code>NLTK</code>

In [8]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import requests
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Amin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Amin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
stop_words = set(stopwords.words('german'))
stemmer = SnowballStemmer("german")

In [63]:
def scrape_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []

    titles = soup.find_all('h1', class_='article-title')
    content_sections = soup.find_all('div', class_='font-m')


    for content in content_sections:
        article = {
            'Titel': [title.get_text(strip=True) for title in titles],
            'Content': content.get_text(strip=True)
        }

        articles.append(article)

    return articles

In [64]:
url = 'https://www.daikin-hbl.de/de/more/content/f%C3%BCchse-verpflichten-linksau%C3%9Fen-aitor-arino-kommt-zur-neuen-saison-nach-berlin-1'
articles = scrape_article(url)
df = pd.DataFrame(articles)
df.to_csv('arino-fuechse.csv', index=False, encoding='utf-8')
df.head()

,Titel,Content
0,[Füchse verpflichten Linksaußen: Aitor Arino k...,Aitor Arino wechselt im Sommer vom FC Barcelon...
1,[Füchse verpflichten Linksaußen: Aitor Arino k...,Da Jerry Tollbring die Füchse aus privater Ent...
2,[Füchse verpflichten Linksaußen: Aitor Arino k...,"Arino und der FC Barcelona, das ist eine ganz ..."
3,[Füchse verpflichten Linksaußen: Aitor Arino k...,Mit Spanien wurde der 89-malige Nationalspiele...
4,[Füchse verpflichten Linksaußen: Aitor Arino k...,Aitor Arino: „Ich wollte noch einmal in einem ...


0    Aitor Arino wechselt im Sommer vom FC Barcelon...
Name: Content, dtype: object


In [52]:
url = 'https://www.daikin-hbl.de/de/hbl'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

articles = []

#titles = soup.find_all('div data-v-94ebeefc', class_='v-card-title text-h5')
titles = soup.find_all('div',class_='v-card-title', attrs={'data-v-94ebeefc':True})
print(titles)

[]


<div class="alert alert-info"><h3> <code>.csv</code>-Datei und DataFrame erfolgreich erstellt (Anhand Titelseite)</h3>
<span><b> Versuche nun, direkt auf die API zuzugreifen und alle Inhalte auf einmal zu extrahieren

In [53]:
base_url = 'https://daikin-hbl.de/api/filter'

params = {
    'type': 'news-list', 
    'id': 57765,
    'page': 2
}

all_articles = []
max_pages = 8

while params['page'] <= max_pages:
    print(f"rufe News Seite {params['page']} ab...")

    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        print(f"Error! Statuscode: {response.status_code} bei Seite {params['page']}")
        break

    data = response.json()
    articles = data.get('articles', [])
    if not articles:
        print(f'keine weiteren Artikel gefunden')
        break
    all_articles.extend(articles)

    params['page'] += 1
    time.sleep(4)

df = pd.DataFrame(all_articles)
df.head()

rufe News Seite 2 ab...
Error! Statuscode: 404 bei Seite 2


""


### Test mit `Selenium`

""
